# Scattering resonances for a circular metamaterial cavity

The codes below are associated to the article:

- C. Carvalho and Z. Moitier, _Asymptotics for metamaterial cavities and their effect on scattering_ [[arXiv](https://arxiv.org/abs/2010.07583), [HAL](https://hal.archives-ouvertes.fr/hal-02965993)]

We present computations related to Section 3 and Appendix C of the manuscript.

## Zoïs Moitier, Camille Carvalho (2021)
            
_Karlsruhe Institute of Technology, Germany_

_University of California Merced, USA_

In [1]:
from sys import argv
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize, SymLogNorm

from latex_plot import set_rcParams, set_size
from zNumeric import *

# Scattering resonances

In [2]:
def plotCX(m, vm, re, im, marker, name):
    ind = vm == m
    plt.plot(re[ind], im[ind], marker, label=name)
    
def m_λ(data):
    return (data[:, 0], (data[:, 1] + 1j * data[:, 2]) ** 2)

def filter_out(res):
    ind = np.where(np.less(np.imag(res[1]), 0))
    return (res[0][ind], res[1][ind])

def plot_scatter(ax, m, λ, M, lw, marker, zorder, label):
    ind = np.where(np.less_equal(m, M))
    sc = ax.scatter(
        np.real(λ[ind]),
        np.imag(λ[ind]),
        s=15,
        lw=lw,
        c=m[ind],
        marker=marker,
        zorder=zorder,
        vmin=0,
        vmax=M,
        label=label,
    )
    return sc

def plot_λ(ε, M, xlim, ylim, xylabel, ax):
    η = np.sqrt(-ε)
    data = np.load(f"../data/eps_{ε}.npz")

    m_out, λ_out = m_λ(data["outer"])
    m_inn, λ_inn = m_λ(data["inner"])
    m_pla, λ_pla = m_λ(data["plasmon"])

    plt.rcParams.update(set_rcParams(font_size=20, line_width=1)) 

    _ = plot_scatter(ax, m_out, λ_out, M, 0, "v", 2, r"$\mathcal{R}_{\mathsf{out}}$")
    sc_inn = plot_scatter(
        ax, m_inn, λ_inn, M, 0.1, ".", 3, r"$\mathcal{R}_{\mathsf{inn}}$"
    )
    _ = plot_scatter(ax, m_pla, λ_pla, M, 0.5, "+", 4, r"$\mathcal{R}_{\mathsf{pla}}$")

    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)

    if xylabel:
        ax.set_xlabel(r"$\Re(\ell^2)$")
        ax.set_ylabel(r"$\Im(\ell^2)$")

    ax.grid(True, zorder=1)
    ax.legend(
        loc="lower center",
        bbox_to_anchor=(0.5, 1.025),
        ncol=3,
        borderaxespad=0.0,
        handlelength=0,
        markerscale=2,
    )
    return sc_inn

εcav = [-1.5,-1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7] 
from ipywidgets import interact 
@interact
def interact_plot_resonances(M:(1, 64, 6)=64, ε1 = εcav, ε2 = εcav):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5), constrained_layout=True)
    sc1 = plot_λ(ε1, M, (-300, 110), (-250, 250 * 0.05), True, ax1)
    sc2 = plot_λ(ε2, M, (-300, 110), (-250, 250 * 0.05), True, ax2)
    cbar = fig.colorbar(sc1, ax=ax1)
    cbar.set_label(r"$m$")
    cbar = fig.colorbar(sc2, ax=ax2)
    cbar.set_label(r"$m$")
    plt.suptitle(fr'Resonances for $\varepsilon$={ε1} and $\varepsilon$={ε2}')

interactive(children=(IntSlider(value=64, description='M', max=64, min=1, step=6), Dropdown(description='ε1', …

# Resonant modes

In [3]:
N = 512
H = 1.5
x = np.linspace(-H, H, num=N)
X, Y = np.meshgrid(x, x)
from ipywidgets import interact 
@interact
def interact_plot_mode(ε = εcav, m:(1, 64, 1)=6, q:(0,20,1)=0):
    η = ε_to_η(ε)
    data = np.load(f"../data/eps_{ε}.npz")

    Int = data["inner"]
    Im = Int[:, 0].astype(int)
    Ir = Int[:, 1]
    Ii = Int[:, 2]

    Pla = data["plasmon"]
    Pm = Pla[:, 0].astype(int)
    Pr = Pla[:, 1]
    Pi = Pla[:, 2]

    Out = data["outer"]
    Om = Out[:, 0].astype(int)
    Or = Out[:, 1]
    Oi = Out[:, 2]
    
    indI = np.where(Im == abs(m))[0]
    indP = np.where(Pm == abs(m))[0]
    indO = np.where(Om == abs(m))[0]

    kI = complex(Ir[min(q, len(indI))], Ii[min(q, len(indI))])
    kP = complex(Pr[min(q, len(indP))], Pi[min(q, len(indP))])
    kO = complex(Or[min(q, len(indO))], Oi[min(q, len(indO))])
    
    f0 = det_M0(η, m)
    
    UI = np.real(res_field_xy(η, m, kI, X, Y))
    UP = np.real(res_field_xy(η, m, kP, X, Y))
    UO = np.real(res_field_xy(η, m, kO, X, Y))
    
    plt.rcParams.update(set_rcParams(font_size=20, line_width=1))
    
    fig, ax = plt.subplots(1,1, figsize = (5,5))
    scO = ax.scatter(
        np.real(kO),
        np.imag(kO),
        s=55,
        lw=1,
        marker='v',
        zorder=2,
        vmin=0,
        vmax=m,
        label= r"$k_{\mathsf{out}}$",
    )
    scI = ax.scatter(
        np.real(kI),
        np.imag(kI),
        s=55,
        lw=1,
        marker='.',
        zorder=3,
        vmin=0,
        vmax=m,
        label= r"$k_{\mathsf{inn}}$",
    )
    scP = ax.scatter(
        np.real(kP),
        np.imag(kP),
        s=55,
        lw=1,
        marker='+',
        zorder=4,
        vmin=0,
        vmax=m,
        label= r"$k_{\mathsf{pla}}$",
    )
    plt.suptitle(fr'Scattering resonances $k$',y=1.1)
    
    ax.grid(True, zorder=1)
    ax.legend(
        loc="lower center",
        bbox_to_anchor=(0.5, 1.025),
        ncol=3,
        borderaxespad=0.0,
        handlelength=0,
        markerscale=2,
    )
    ax.set_xlim(-300, 110)
    ax.set_ylim(-250, 100)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (30,10))
    Urmax1 = np.abs(UP).max()
    norm1 = Normalize(vmin=-Urmax1, vmax=Urmax1)
    ticks1 = [-1, -0.5, 0, 0.5, 1]
    im1 = ax1.pcolormesh(X, Y, np.real(UP), shading="auto", cmap="RdBu_r", norm=norm1)
    ax1.set_aspect("equal", "box")
    cbar = fig.colorbar(im1, ax=ax1, ticks=ticks1)
    cbar.ax.tick_params(which="minor", length=0)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title(f'Plasmonic mode for m={m}, q={min(q, len(indP))}')
    
    Urmax2 = np.abs(UI).max()
    norm2 = Normalize(vmin=-Urmax2, vmax=Urmax2)
    ticks2 = [-2, -1, 0, 1, 2]
    im2 = ax2.pcolormesh(X, Y, np.real(UI), shading="auto", cmap="RdBu_r", norm=norm2)
    ax2.set_aspect("equal", "box")
    cbar = fig.colorbar(im2, ax=ax2, ticks=ticks2)
    cbar.ax.tick_params(which="minor", length=0)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title(f'Inner mode for m={m}, q={min(q, len(indI))}')
    
    Urmax3 = np.abs(UO).max()
    norm3 = SymLogNorm(1, vmin=-Urmax3, vmax=Urmax3, base=10)
    ticks3 = [-1e6, -1e4, -1e2, -1, 0, 1, 1e2, 1e4, 1e6]
    im3 = ax3.pcolormesh(X, Y, np.real(UO), shading="auto", cmap="RdBu_r", norm=norm3)
    ax3.set_aspect("equal", "box")
    cbar = fig.colorbar(im3, ax=ax3, ticks=ticks3)
    cbar.ax.tick_params(which="minor", length=0)
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title(f'Outer mode for m={m}, q={min(q, len(indO))}')
    


interactive(children=(Dropdown(description='ε', options=(-1.5, -1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7), val…